In [ ]:
%%capture #usually run on colab so I installed all the packages here
!pip install torch transformers
!pip install -U datasets
!pip install tqdm
!pip install -U bitsandbytes

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from datasets import load_dataset
from huggingface_hub import notebook_login
#notebook_login()

In [ ]:
import pandas as pd


In [ ]:
#df = pd.read_csv('/content/drive/MyDrive/SYNTHETIC DATA/synthetic_data.csv')

In [ ]:
ds = load_dataset("hrithikpiyush/acl-arc")
ds.set_format("pandas")
train=ds["train"][:]
validation=ds["validation"][:]
test=ds["test"][:]


In [ ]:
# prompt: add df to train and fill everything else with NaN

# Concatenate the new dataframe with the train dataframe
# We use ignore_index=True to reset the index of the combined dataframe
#train = pd.concat([train, df], ignore_index=True)

In [ ]:
# prompt: save train as "syntheticdataset.csv"

#train.to_csv("/content/drive/MyDrive/ARCACL/syntheticdataset.csv", index=False)
#train

In [ ]:
# prompt: count the number of train['intent']==5

intent_5_count = train['intent'].value_counts().get(5, 0)
print(f"Number of rows where 'intent' is 5: {intent_5_count}")


Number of rows where 'intent' is 5: 60


In [ ]:
# Charger le modèle Llama-3.2-1B-Instruct
#model_name = "meta-llama/Llama-3.2-1B-Instruct"
model_name="Qwen/Qwen3-14B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, output_hidden_states=True,load_in_8bit=True)



# train activations

(redo that with validation and test by changing names (to avoid code bloat))

In [ ]:
from tqdm.notebook import tqdm
from huggingface_hub import HfApi

#INITIALISATION
#liste_token_citation=[]


liste_last_token=[torch.empty((0, 5120), device=model.device)]*40
#liste_last_token=[torch.empty((0, hidden_dim), device=model.device)]*40
# for layer in range(len(model.model.layers)):
#   #liste_token_citation.append([])
#   liste_last_token.append([])




for i,text in tqdm(enumerate(train['cleaned_cite_text'][:])): #pour chaque texte #REMPLACER LE DATASET !!
  inputs = tokenizer(text, return_tensors="pt")
  inputs = {k: v.to(model.device) for k, v in inputs.items()}
  with torch.no_grad():
      outputs = model(**inputs)
  hidden_states = outputs.hidden_states
  # Passer le texte dans le modèle pour récupérer les activations

  for layer in range(0,len(model.model.layers)): #pour chaque 4 layers
    layer_index = layer
    layer_activation = hidden_states[layer_index]  # (batch_size, seq_len, hidden_dim)



    #cls_representation = layer_activation[:, position_token[i]+3, :]  # on récupère le ATION de @@CITATION
    #liste_token_citation[layer].append(cls_representation) #on le rajoute dans la bonne liste
    last_token = layer_activation[:, -1, :] #on récupère le dernier token
    #print(liste_last_token[layer].shape)
    last_token=torch.nn.functional.normalize(last_token, p=2, dim=0) # On normalise !
    liste_last_token[layer]=torch.cat((liste_last_token[layer].cpu(), last_token.cpu()), dim=0).to('cpu')
    #liste_last_token[layer].append(last_token)  # (1, hidden_dim)

#sauvegarder chaque elt de la liste dans un fichier
api = HfApi()
for i in range(len(liste_last_token)):
  torch.save(liste_last_token[i], '/content/synthetic_train_layer_'+str(i)+'.pt')
  api.upload_file(path_or_fileobj='synthetic_train_layer_'+str(i)+'.pt', path_in_repo='train_normalized/synthetic_train_layer_'+str(i)+'.pt', repo_id="#ANONYMIZED/Qwen3Activations", repo_type="dataset", )
  #torch.save(liste_token_citation[i], '/content/drive/MyDrive/ACTIVATION_VALIDATION/layer_'+str(i)+'_train_@@token.pt')
  #torch.save(liste_last_token[i], '/content/drive/MyDrive/A NEW QWEN/AMAIN_TASK_QWEN/synthetic_train_layer_'+str(i)+'__last_token.pt')